In [25]:
# !pip install selenium
# !pip install webdriver-manager
# View your Google Chrome browser version: chrome://settings/help
# Download ChromeDriver that corresponds to your Google Chrome browser version: https://sites.google.com/chromium.org/driver/

Useful links:
https://stackoverflow.com/questions/69875125/find-element-by-commands-are-deprecated-in-selenium

In [26]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [27]:
CHROME_DRIVER_PATH = "C:/Users/ernes/Downloads/chromedriver_win32/chromedriver.exe"

In [28]:
ser = Service(CHROME_DRIVER_PATH)
op = webdriver.ChromeOptions()

# Selenium Tutorial
https://www.youtube.com/playlist?list=PLzMcBGfZo4-n40rB1XaJ0ak1bemvlqumQ

In [29]:
# This cell will open a Chrome browser. DO NOT CLOSE IT MANUALLY. run: driver.quit() to close it instead
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://techwithtim.net")

In [30]:
# Finding search bar
search = driver.find_element(By.NAME, "s")
search.send_keys("test")
search.send_keys(Keys.RETURN)

# Waiting for 10 seconds after sending keys to search bar
try:
    main = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "main"))
    )
except:
    driver.quit()

In [31]:
# Looking for multiple elements from main page
articles = main.find_elements(By.TAG_NAME, "article")
for article in articles:
    header = article.find_element(By.CLASS_NAME, "entry-summary")
    print(header.text)

HTTP MethodsIn this tutorial we will talk about HTTP methods. HTTP methods are the standard way of sending information to and from a web server. To break it down, a website runs on a server or multiple servers and simple returns information to a client (web-browser). Information is exchanged between the client and the server […]
Creating a Base Template So you may have realized that creating new web pages for every single page on our website is extremely inefficient. Especially when our website follows a theme and has similar elements (like a sidebar) on every page. This is where template inheritance comes in. We will talk about how to inherit […]
Redirecting ContinuedStarting from where we left off in the last tutorial. I wanted to show how to redirect to a function that takes an argument (like our user function). To do this we simply need to define the parameter name and a value in the url_for function, like below.from flask import Flask, redirect, url_for app […]
What is Flask?Flask

In [41]:
# Going back one page
driver.back()

In [42]:
link = driver.find_element(By.LINK_TEXT, "Python Programming")
link.click()

# Waiting for 10 seconds after sending keys to search bar
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.LINK_TEXT, "Beginner Python Tutorials"))
    )
    element.click()
    
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "sow-button-19310003"))
    )
    element.click()
except:
    driver.quit()

In [43]:
driver.quit()

# JobStreet Scraping

In [99]:
website = "https://www.jobstreet.com.sg/"
driver = webdriver.Chrome(service=ser, options=op)
driver.get(website)

In [100]:
# Finding search bar
search = driver.find_element(By.ID, "searchKeywordsField")
search.send_keys("data scientist")
search.send_keys(Keys.RETURN)

In [104]:
jobs = driver.find_element(By.CLASS_NAME, "sx2jih0")

In [108]:
test = driver.find_element(By.CLASS_NAME, "sx2jih0.zcydq8bm")

<selenium.webdriver.remote.webelement.WebElement (session="57a5b6498c6837930d71834919972766", element="6fc2591c-ed1e-4faa-bbad-e3abfbd1dd0a")>

In [109]:
driver.quit()